# Milestone 1:  Circuit and Simulation in SNAP

### Setup


In [ ]:
import getpass
import logging
from pathlib import Path

from bluepysnap.api import Api

logging.basicConfig(level=logging.WARNING)

ORGANIZATION = "nse"
PROJECT = "test"

Paste the token from https://bbp.epfl.ch/nexus/web/

In [ ]:
TOKEN = getpass.getpass()

In [ ]:
nexus_config = "../../../examples/configurations/api/forge.yml"
api = Api(nexus_config=nexus_config, bucket=f"{ORGANIZATION}/{PROJECT}", token=TOKEN, debug=False)

### Instantiate a SNAP circuit from a KG URL

In [ ]:

circuit_id = "https://bbp.epfl.ch/nexus/v1/resources/nse/test/_/20191025"
circuit = api.get_entity_by_id(circuit_id)
print(circuit.type, circuit.id)
print(circuit)

In [ ]:
print(circuit.resource)

In [ ]:
print(circuit.instance)

### Illustrate bluepy/snap with a small piece of code

In [ ]:
from bluepysnap.bbp import Cell
from matplotlib import pyplot as plt

cells = circuit.instance.nodes.get(properties=[Cell.X, Cell.Y, Cell.Z])

In [ ]:
plt.plot(cells[Cell.X], cells[Cell.Y], ".b")

### Get information about the circuit

In [ ]:
circuit.description

In [ ]:
# FIXME: shouldn't be circuit.brainRegion, according to neuroshapes.org?
region = circuit.brainLocation.brainRegion
print(region.id)
print(region.label)

In [ ]:
# FIXME: is species optional? How to handle missing optional attributes?
species = getattr(circuit, 'species', None)
print(species)

In [ ]:
# FIXME: what do we mean by project in this context? test, nse/test, https://bbp.epfl.ch/nexus/v1/projects/nse/test
print(circuit.project)

In [ ]:
# FIXME: according to neuroshapes.org, the entity could have a dateCreated attribute. Should it be considered?
print(circuit.createdBy)
print(circuit.createdAt)

### Get Simulations that used that circuit

In [ ]:
# simulations that used the circuit
sim1 = api.connector.get_resources(
    "Simulation",
    {"used": f"<{circuit.id}>"}
)

# simulation campaigns that used the circuit
simulation_campaigns = api.connector.get_resources(
    "SimulationCampaign",
    {"used": f"<{circuit.id}>"}
)

# simulations started by the simulation campaigns
ids = [f"<{r.id}>" for r in simulation_campaigns]
sim2 = api.connector.get_resources(
    "Simulation",
    {"wasStartedBy": ids}
)

# merge simulations and remove possible duplicates
simulations = {s.id: s for s in sim1 + sim2}
simulations = list(simulations.values())
print(len(sim1), len(sim2), len(simulations))

In [ ]:
# alternatively, we provide a ready-to-use (but less flexible) function
query = f"""
    SELECT DISTINCT ?id
    WHERE {{
        {{
            ?id a Simulation ; wasStartedBy ?scid .
            ?scid a SimulationCampaign ; used <{circuit.id}> .
        }}
    UNION
        {{
            ?id a Simulation ; used <{circuit.id}> .
        }}
    }}
    """
simulations2 = api.connector.get_resources_by_query(query)
sorted(r.id for r in simulations) == sorted(r.id for r in simulations2)


In [ ]:
# FIXME: alternatively, we provide a ready-to-use (but less flexible) function
simulation_entities = api.simulation.get_simulations_by_circuit(circuit, tool='bluepy')

sorted(r.id for r in simulations) == sorted(r.id for r in simulation_entities)


### Instantiate these simulation as ~~SNAP~~ BLUEPY entities either from the circuit or from a KG URL

In [ ]:
# not needed if using rentity_simulations from the previous step
simulation_entities2 = [api.factory.open(r, "bluepy") for r in simulations]

### For each simulation display information about the simulation
- Configuration of the simulation 
- Execution time / Creator
- KG Project it belongs to
- Description

### and Display a spike raster and psth

In [ ]:

for n, sim in enumerate(simulation_entities):
    print(f"### Simulation {n}")
    print("Simulation path:", sim.path)
    print("createdAt:", sim.createdAt)
    print("createdBy:", sim.createdBy)
    print("project:", sim.project)
    print("description:", sim.description)
    sim.instance.plot.spikeraster_and_psth('soma', ['Mosaic'])
    break
